<a href="https://colab.research.google.com/github/tarunpanda001/Machine_learning_Models/blob/main/GradientBoosting_DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
!gdown 16KtxSt_QEGQvfluEaMls5cCHPwhRXgCk

Downloading...
From: https://drive.google.com/uc?id=16KtxSt_QEGQvfluEaMls5cCHPwhRXgCk
To: /content/HR-Employee-Attrition.csv
100% 228k/228k [00:00<00:00, 5.68MB/s]


In [6]:
df=pd.read_csv("HR-Employee-Attrition.csv")
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [9]:
df.shape

(1470, 35)

In [10]:
# These columns do not provide predictive power
df = df.drop(['EmployeeCount', 'Over18', 'StandardHours', 'EmployeeNumber'], axis=1)

In [11]:
# Encoding Categorical Columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cat_cols=df.select_dtypes(include="object").columns
for col in cat_cols:
  df[col] = le.fit_transform(df[col])

In [12]:
 X=df.drop("Attrition",axis=1)
 y=df["Attrition"]

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
from imblearn.over_sampling import SMOTE

smote=SMOTE()
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [33]:
from sklearn.tree import DecisionTreeClassifier as dtc
model=dtc(max_depth=2, max_leaf_nodes=60)

In [34]:
model.fit(X_train_res,y_train_res)

DecisionTreeClassifier(max_depth=2, max_leaf_nodes=60)

In [35]:
model.score(X_train_res,y_train_res)

0.7438650306748467

In [36]:
model.score(X_test,y_test)

0.6836734693877551

In [58]:
from sklearn.metrics import accuracy_score
y_pred=model.predict(X_test)
print(accuracy_score(y_test,y_pred))


0.6836734693877551


## Since the current model is clearly underfitting, we will use Gradient Boosting Decision Trees to improve its performance.

## Implementation Of Gradient Boosting Decision Trees

In [37]:
params={
    "n_estimators":[5,50,250,500],
    "max_depth":[1,3,5,7,9],
    "learning_rate":[0.01,0.1,1,10],
    "learning_rate":[0.1,0.2,0.3]
}

In [40]:
from sklearn.ensemble import GradientBoostingClassifier as gbc
from sklearn.model_selection import RandomizedSearchCV


final_model=gbc()
clf=RandomizedSearchCV(final_model,params,cv=3,verbose=2,n_jobs=-1)
clf.fit(X_train_res,y_train_res)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=GradientBoostingClassifier(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.1, 0.2, 0.3],
                                        'max_depth': [1, 3, 5, 7, 9],
                                        'n_estimators': [5, 50, 250, 500]},
                   verbose=2)

In [43]:
res = clf.cv_results_

for i in range(len(res["params"])):
  print(f"Parameters:{res['params'][i]} Mean_score: {res['mean_test_score'][i]} Rank: {res['rank_test_score'][i]}")

Parameters:{'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.1} Mean_score: 0.908997955010225 Rank: 1
Parameters:{'n_estimators': 250, 'max_depth': 9, 'learning_rate': 0.2} Mean_score: 0.9043967280163598 Rank: 2
Parameters:{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.2} Mean_score: 0.894683026584867 Rank: 6
Parameters:{'n_estimators': 50, 'max_depth': 7, 'learning_rate': 0.3} Mean_score: 0.9023517382413088 Rank: 3
Parameters:{'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.1} Mean_score: 0.9013292433537833 Rank: 5
Parameters:{'n_estimators': 5, 'max_depth': 9, 'learning_rate': 0.3} Mean_score: 0.8333333333333334 Rank: 8
Parameters:{'n_estimators': 250, 'max_depth': 9, 'learning_rate': 0.1} Mean_score: 0.9023517382413088 Rank: 3
Parameters:{'n_estimators': 50, 'max_depth': 1, 'learning_rate': 0.3} Mean_score: 0.8231083844580778 Rank: 9
Parameters:{'n_estimators': 50, 'max_depth': 9, 'learning_rate': 0.1} Mean_score: 0.8578732106339468 Rank: 7
Parameters:{'n_est

In [44]:
print(clf.best_estimator_)

GradientBoostingClassifier(max_depth=7, n_estimators=500)


In [51]:
final_model = clf.best_estimator_

final_model.fit(X_train_res, y_train_res)

GradientBoostingClassifier(max_depth=7, n_estimators=500)

In [52]:
final_model.score(X_train_res, y_train_res)

1.0

In [53]:
final_model.score(X_test,y_test)

0.8503401360544217

In [60]:
from sklearn.metrics import accuracy_score
y_pred=final_model.predict(X_test)
print(accuracy_score(y_test,y_pred))


0.8503401360544217
